# import important libraries

In [2]:

import warnings
warnings.filterwarnings("ignore")

# ================================
# 1. Imports
# ================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

plt.style.use("default")
pd.set_option("display.max_columns", None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

try:
    from imblearn.over_sampling import SMOTE
    HAS_SMOTE = True
except ImportError:
    print("⚠️ imblearn not installed. SMOTE will be skipped.")
    HAS_SMOTE = False





# Transactions_data Cleaning And Analysis

In [3]:
df = pd.read_csv("transactions_data.csv" , parse_dates=["date"])
df  # convert datatype of date to datetime

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,NaN
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,5815,NaN
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,NaN
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,NaN


In [4]:
df.head(10) # Data Sorted By ID , Date  , merbahnt City (online) ----> State (NaN) , ZIP (NaN)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
5,7475333,2010-01-01 00:07:00,1807,165,$4.81,Swipe Transaction,20519,Bronx,NY,10464.0,5942,NaN
6,7475334,2010-01-01 00:09:00,1556,2972,$77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
7,7475335,2010-01-01 00:14:00,1684,2140,$26.46,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,$261.58,Online Transaction,50292,ONLINE,NaN,NaN,7801,NaN
9,7475337,2010-01-01 00:21:00,351,1112,$10.74,Swipe Transaction,3864,Flushing,NY,11355.0,5813,NaN


In [5]:
df.tail()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,NaN
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,5815,NaN
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,NaN
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,NaN
13305914,23761874,2019-10-31 23:59:00,489,5697,$12.88,Chip Transaction,24658,Greenbrier,TN,37073.0,5921,NaN


In [6]:
df.info() # row (date) converted to datatime

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          object        
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             float64       
 10  mcc             int64         
 11  errors          object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.2+ GB


In [7]:
df[df["amount"].str.contains("$")] # Amount Should Be converted To float 

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,NaN
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,5815,NaN
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,NaN
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,NaN


In [8]:
df['amount'] = df['amount'].str.replace('$', '', regex=False).astype(float) # convert Amount To Float

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             float64       
 10  mcc             int64         
 11  errors          object        
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 1.2+ GB


In [10]:
df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1563700
zip                1652706
mcc                      0
errors            13094522
dtype: int64

In [11]:
NAN_state = df[df["merchant_state"].isna()]
NAN_state

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,26.46,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,261.58,Online Transaction,50292,ONLINE,NaN,NaN,7801,NaN
18,7475346,2010-01-01 00:34:00,394,4717,26.04,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
24,7475353,2010-01-01 00:43:00,301,3742,10.17,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
26,7475356,2010-01-01 00:45:00,566,3439,16.86,Online Transaction,16798,ONLINE,NaN,NaN,4121,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305879,23761832,2019-10-31 23:22:00,1556,2972,17.65,Online Transaction,88459,ONLINE,NaN,NaN,5311,NaN
13305880,23761833,2019-10-31 23:22:00,1797,5660,34.81,Online Transaction,15143,ONLINE,NaN,NaN,4784,NaN
13305888,23761843,2019-10-31 23:33:00,1069,5167,59.71,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
13305897,23761853,2019-10-31 23:39:00,1422,5696,694.30,Online Transaction,70268,ONLINE,NaN,NaN,4722,NaN


In [12]:
NAN_state["use_chip" ].unique() 

array(['Online Transaction', 'Chip Transaction'], dtype=object)

In [13]:
NAN_state["merchant_city" ].unique() # merchant_city ('Online')

array(['ONLINE'], dtype=object)

In [14]:
NAN_state[NAN_state["use_chip"] == 'Chip Transaction' ] # merchant_city ('Online')

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
6573529,15473515,2015-01-01 12:19:00,944,2828,-134.00,Chip Transaction,13348,ONLINE,NaN,NaN,4722,NaN
6573565,15473561,2015-01-01 12:27:00,944,2828,358.94,Chip Transaction,13348,ONLINE,NaN,NaN,4722,NaN
6573594,15473593,2015-01-01 12:32:00,1117,1162,117.43,Chip Transaction,52073,ONLINE,NaN,NaN,4722,NaN
6573677,15473695,2015-01-01 12:51:00,1117,1162,-147.00,Chip Transaction,52073,ONLINE,NaN,NaN,4722,NaN
6574120,15474225,2015-01-01 14:24:00,972,6052,233.77,Chip Transaction,13348,ONLINE,NaN,NaN,4722,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13301848,23756824,2019-10-30 22:43:00,383,3686,160.82,Chip Transaction,90999,ONLINE,NaN,NaN,4722,NaN
13302226,23757266,2019-10-31 06:15:00,46,3513,-387.00,Chip Transaction,90999,ONLINE,NaN,NaN,4722,NaN
13302233,23757275,2019-10-31 06:16:00,46,3513,208.11,Chip Transaction,90999,ONLINE,NaN,NaN,4722,NaN
13305528,23761384,2019-10-31 20:03:00,1170,4592,129.47,Chip Transaction,13348,ONLINE,NaN,NaN,4722,NaN


In [15]:
df["merchant_state"].fillna("ONLINE" , inplace= True) # The Right Value By logic

In [16]:
df.isna().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state           0
zip                1652706
mcc                      0
errors            13094522
dtype: int64

In [17]:
df["errors"].fillna("No Erros" , inplace=True) # The Right Value By logic

In [18]:
NAN_Zip = df[df["zip"].isna()] 
NAN_Zip

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,26.46,Online Transaction,39021,ONLINE,ONLINE,NaN,4784,No Erros
8,7475336,2010-01-01 00:21:00,335,5131,261.58,Online Transaction,50292,ONLINE,ONLINE,NaN,7801,No Erros
18,7475346,2010-01-01 00:34:00,394,4717,26.04,Online Transaction,39021,ONLINE,ONLINE,NaN,4784,No Erros
24,7475353,2010-01-01 00:43:00,301,3742,10.17,Online Transaction,39021,ONLINE,ONLINE,NaN,4784,No Erros
26,7475356,2010-01-01 00:45:00,566,3439,16.86,Online Transaction,16798,ONLINE,ONLINE,NaN,4121,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
13305879,23761832,2019-10-31 23:22:00,1556,2972,17.65,Online Transaction,88459,ONLINE,ONLINE,NaN,5311,No Erros
13305880,23761833,2019-10-31 23:22:00,1797,5660,34.81,Online Transaction,15143,ONLINE,ONLINE,NaN,4784,No Erros
13305888,23761843,2019-10-31 23:33:00,1069,5167,59.71,Online Transaction,39021,ONLINE,ONLINE,NaN,4784,No Erros
13305897,23761853,2019-10-31 23:39:00,1422,5696,694.30,Online Transaction,70268,ONLINE,ONLINE,NaN,4722,No Erros


In [19]:
NAN_Zip["merchant_city"].unique()

array(['ONLINE', 'Puerto Vallarta', 'Vatican City', 'Guadalajara',
       'Santo Domingo', 'Montreal', 'Toronto', 'San Jose', 'Berlin',
       'Mexico City', 'Shanghai', 'Cancun', 'Edinburgh', 'Tallinn',
       'Funafuti', 'Tapei', 'Abu Dhabi', 'Beijing', 'Vilnius',
       'Amsterdam', 'Tokyo', 'Athens', 'Ho Chi Minh City',
       'Port au Prince', 'Dublin', 'Singapore', 'Paris', 'Johannesberg',
       'Cabo San Lucas', 'Bangkok', 'Rome', 'Copenhagen', 'Kingston',
       'Porto-Novo', 'London', 'Edmonton', 'Calgary', 'Brussels',
       'Freetown', 'Jakarta', 'Bogota', 'Geneva', 'Vancouver', 'Lisbon',
       'Wellington', 'Amman', 'Guatamala City', 'Hong Kong', 'Helsinki',
       'Ulan Bator', 'Riyadh', 'Manila', 'Oslo', 'Budapest', 'Seoul',
       'Jerusalem', 'Mumbai', 'Sydney', 'Monaco', 'Bucharest',
       'Saint Petersburg', 'Vienna', 'Majuro', 'Delhi', 'Barcelona',
       'Zurich', 'Oranjestad', 'Lima', 'Chisinau', 'Acapulco', 'Dhaka',
       'Madrid', 'Sao Paolo', 'Palikir', 'Tas

In [20]:
NAN_Zip["merchant_state"].unique()

array(['ONLINE', 'Mexico', 'Vatican City', 'Dominican Republic', 'Canada',
       'Costa Rica', 'Germany', 'China', 'United Kingdom', 'Estonia',
       'Tuvalu', 'Taiwan', 'United Arab Emirates', 'Lithuania',
       'Netherlands', 'Japan', 'Greece', 'Vietnam', 'Haiti', 'Ireland',
       'Singapore', 'France', 'South Africa', 'Thailand', 'Italy',
       'Denmark', 'Jamaica', 'Benin', 'Belgium', 'Sierra Leone',
       'Indonesia', 'Colombia', 'Switzerland', 'Portugal', 'New Zealand',
       'Jordan', 'Guatemala', 'Hong Kong', 'Finland', 'Mongolia',
       'Saudi Arabia', 'Philippines', 'Norway', 'Hungary', 'South Korea',
       'Israel', 'India', 'Australia', 'Monaco', 'Romania', 'Russia',
       'Austria', 'Marshall Islands', 'Spain', 'Aruba', 'Peru', 'Moldova',
       'Bangladesh', 'Brazil', 'Micronesia', 'Uzbekistan', 'Chile',
       'Croatia', 'Sweden', 'Brunei', 'Antigua and Barbuda', 'Andorra',
       'Kenya', 'Czech Republic', 'Tonga', 'Eritrea', 'Solomon Islands',
       'Pakista

In [21]:
df.loc[df["merchant_city"] == "ONLINE" , "zip"] = 0 # The Right Value By logic

In [22]:
df.isna().sum()

id                    0
date                  0
client_id             0
card_id               0
amount                0
use_chip              0
merchant_id           0
merchant_city         0
merchant_state        0
zip               89006
mcc                   0
errors                0
dtype: int64

In [23]:
df[(df["merchant_city"] == "Puerto Vallarta") & (df["merchant_state"] == "Mexico")]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
581,7476010,2010-01-01 07:53:00,1579,3830,6.51,Swipe Transaction,22204,Puerto Vallarta,Mexico,NaN,5541,No Erros
3920,7479985,2010-01-02 07:43:00,1579,11,7.38,Swipe Transaction,7886,Puerto Vallarta,Mexico,NaN,5812,No Erros
5559,7481955,2010-01-02 15:59:00,1579,11,13.23,Swipe Transaction,1344,Puerto Vallarta,Mexico,NaN,5411,No Erros
6980,7483640,2010-01-03 07:33:00,1579,2170,2.86,Swipe Transaction,7886,Puerto Vallarta,Mexico,NaN,5812,No Erros
7036,7483705,2010-01-03 07:45:00,1579,11,95.05,Swipe Transaction,61195,Puerto Vallarta,Mexico,NaN,5541,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
13243107,23684416,2019-10-15 11:55:00,756,3963,71.00,Chip Transaction,59935,Puerto Vallarta,Mexico,NaN,5499,No Erros
13247517,23689850,2019-10-16 15:52:00,756,3963,-347.00,Chip Transaction,66954,Puerto Vallarta,Mexico,NaN,7011,No Erros
13247530,23689866,2019-10-16 15:54:00,756,3963,489.24,Swipe Transaction,66954,Puerto Vallarta,Mexico,NaN,7011,No Erros
13248172,23690670,2019-10-16 19:47:00,756,3963,-472.00,Chip Transaction,51300,Puerto Vallarta,Mexico,NaN,3359,No Erros


In [24]:
df["zip"][(df["merchant_city"] == "Puerto Vallarta") & (df["merchant_state"] == "Mexico")].unique()

array([nan])

In [25]:
# The Right Value By logic And Searchig
missing_zip = {
  "Puerto Vallarta": "48300",
  "Vatican City": "00120",
  "Guadalajara": "44100",
  "Santo Domingo": "10101",
  "Montreal": "H3A",
  "Toronto": "M5H",
  "San Jose": "10101",
  "Berlin": "10115",
  "Mexico City": "01000",
  "Shanghai": "200000",
  "Cancun": "77500",
  "Edinburgh": "EH1",
  "Tallinn": "10111",
  "Funafuti": "",
  "Tapei": "100",
  "Abu Dhabi": "00000",
  "Beijing": "100000",
  "Vilnius": "01100",
  "Amsterdam": "1011",
  "Tokyo": "100-0001",
  "Athens": "10552",
  "Ho Chi Minh City": "700000",
  "Port au Prince": "6110",
  "Dublin": "D01",
  "Singapore": "018989",
  "Paris": "75001",
  "Johannesberg": "",
  "Cabo San Lucas": "23450",
  "Bangkok": "10200",
  "Rome": "00184",
  "Copenhagen": "1050",
  "Kingston": "",
  "Porto-Novo": "",
  "London": "SW1A 1AA",
  "Edmonton": "T5J",
  "Calgary": "T2P",
  "Brussels": "1000",
  "Freetown": "",
  "Jakarta": "10110",
  "Bogota": "110111",
  "Geneva": "1201",
  "Vancouver": "V5K",
  "Lisbon": "1100-148",
  "Wellington": "6011",
  "Amman": "11118",
  "Guatamala City": "01001",
  "Hong Kong": "999077",
  "Helsinki": "00100",
  "Ulan Bator": "15160",
  "Riyadh": "12611",
  "Manila": "1000",
  "Oslo": "0150",
  "Budapest": "1051",
  "Seoul": "04524",
  "Jerusalem": "91000",
  "Mumbai": "400001",
  "Sydney": "2000",
  "Monaco": "98000",
  "Bucharest": "010011",
  "Saint Petersburg": "190000",
  "Vienna": "1010",
  "Majuro": "96960",
  "Delhi": "110001",
  "Barcelona": "08001",
  "Zurich": "8001",
  "Oranjestad": "",
  "Lima": "15001",
  "Chisinau": "MD-2001",
  "Acapulco": "39300",
  "Dhaka": "1000",
  "Madrid": "28001",
  "Sao Paulo": "01000-000",
  "Palikir": "96941",
  "Tashkent": "100000",
  "Santiago": "8320000",
  "Zagreb": "10000",
  "Kolkata": "700001",
  "Bangalore": "560001",
  "Rio de Janeiro": "20000-000",
  "Stockholm": "11120",
  "Bandar Seri Begawan": "BA1111",
  "Saint John's": "",
  "Andorra La Vella": "AD500",
  "Nairobi": "00100",
  "Prague": "110 00",
  "Nuku Alofa": "",
  "Asmara": "",
  "Honiara": "",
  "Islamabad": "44000",
  "Nassau": "",
  "Rabat": "10000",
  "Suva": "",
  "Muscat": "113",
  "Yaounde": "",
  "Skopje": "1000",
  "Tegucigalpa": "11101",
  "Abuja": "900001",
  "Belmopan": "",
  "Montevideo": "11000",
  "Istanbul": "34000",
  "Manama": "",
  "Moscow": "101000",
  "Kuala Lumpur": "50000",
  "Luxembourg": "L-1111",
  "Algiers": "16000",
  "Bridgetown": "BB11114",
  "Niamey": "",
  "Cairo": "11511",
  "Hanoi": "100000",
  "Accra": "00233",
  "Bratislava": "811 01",
  "Kiev": "01001",
  "Yaren District": "",
  "Buenos Aires": "C1000",
  "Juba": "",
  "Podgorica": "81000",
  "Colombo": "00100",
  "Lahore": "54000",
  "Reykjavik": "101",
  "Ljubljana": "1000",
  "Sanaa": "",
  "Riga": "LV-1050",
  "Doha": "00000",
  "Karachi": "74000",
  "Male": "20026",
  "Ouagadougou": "",
  "Harare": "",
  "Praia": "",
  "Malabo": "",
  "Port Vila": "",
  "Addis Ababa": "1000",
  "Mbabane": "",
  "Maputo": "1100",
  "Warsaw": "00-001",
  "Lusaka": "10101",
  "Kingstown": "",
  "Tblisi": "0105",
  "Baghdad": "10001",
  "Apia": "",
  "Monrovia": "1000",
  "Bamako": "",
  "Nicosia": "1010",
  "Quito": "170101",
  "Dakar": "",
  "Libreville": "",
  "Belgrade": "11000",
  "Tunis": "1000",
  "Pristina": "10000",
  "Bishek": "720001",
  "Victoria": "",
  "Sarajevo": "71000",
  "Panama City": "0819",
  "Paramaribo": "",
  "Yamoussoukro": "",
  "Yangon": "11181",
  "Tehran": "11369",
  "Beirut": "1107",
  "Valletta": "VLT 1117",
  "Tirana": "1001",
  "Conakry": "",
  "Port Moresby": "111",
  "Dili": "",
  "Caracas": "1010",
  "Khartoum": "11111",
  "Port of Spain": "",
  "Brazzaville": "",
  "Baku": "AZ1000",
  "Georgetown": ""
}

In [26]:
df["zip"] = df["zip"].fillna(df["merchant_city"].map(missing_zip))

In [27]:
df.isna().sum()

id                  0
date                0
client_id           0
card_id             0
amount              0
use_chip            0
merchant_id         0
merchant_city       0
merchant_state      0
zip               434
mcc                 0
errors              0
dtype: int64

In [28]:
df["merchant_city"][df["zip"].isna()].unique()

array(['Sao Paolo'], dtype=object)

In [29]:
df["zip"].fillna("01000-000" , inplace= True)

In [30]:
df.isna().sum()

id                0
date              0
client_id         0
card_id           0
amount            0
use_chip          0
merchant_id       0
merchant_city     0
merchant_state    0
zip               0
mcc               0
errors            0
dtype: int64

In [31]:
df.duplicated().sum()

np.int64(0)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   client_id       int64         
 3   card_id         int64         
 4   amount          float64       
 5   use_chip        object        
 6   merchant_id     int64         
 7   merchant_city   object        
 8   merchant_state  object        
 9   zip             object        
 10  mcc             int64         
 11  errors          object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.2+ GB


In [33]:
df.describe()

,id,date,client_id,card_id,amount,merchant_id,mcc
count,1.330592e+07,13305915,1.330592e+07,1.330592e+07,1.330592e+07,1.330592e+07,1.330592e+07
mean,1.558402e+07,2015-01-06 06:50:27.604354816,1.026812e+03,3.475268e+03,4.297604e+01,4.772376e+04,5.565440e+03
min,7.475327e+06,2010-01-01 00:01:00,0.000000e+00,0.000000e+00,-5.000000e+02,1.000000e+00,1.711000e+03
25%,1.150604e+07,2012-08-08 21:43:00,5.190000e+02,2.413000e+03,8.930000e+00,2.588700e+04,5.300000e+03
50%,1.557087e+07,2015-01-22 12:17:00,1.070000e+03,3.584000e+03,2.899000e+01,4.592600e+04,5.499000e+03
75%,1.965361e+07,2017-06-13 17:45:00,1.531000e+03,4.901000e+03,6.371000e+01,6.757000e+04,5.812000e+03
max,2.376187e+07,2019-10-31 23:59:00,1.998000e+03,6.144000e+03,6.820200e+03,1.003420e+05,9.402000e+03
std,4.704499e+06,NaN,5.816386e+02,1.674356e+03,8.165575e+01,2.581534e+04,8.757002e+02


In [34]:
df[df["amount"] == df["amount"].max()]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
892174,8544734,2010-09-22 06:37:00,708,5165,6820.2,Swipe Transaction,34524,Staten Island,NY,10302.0,5712,No Erros


In [35]:
df[df["client_id"] == 708] # Important client -------> Total amount = 1094355.64

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
1623,7477273,2010-01-01 12:30:00,708,5165,17.29,Swipe Transaction,61195,New York,NY,10075.0,5541,No Erros
1811,7477501,2010-01-01 13:13:00,708,5165,72.58,Swipe Transaction,11368,New York,NY,10069.0,5411,No Erros
4046,7480138,2010-01-02 08:24:00,708,1106,32.03,Swipe Transaction,81829,New York,NY,10075.0,5812,No Erros
4882,7481142,2010-01-02 12:49:00,708,1106,25.11,Swipe Transaction,23481,New York,NY,10075.0,5812,No Erros
5724,7482150,2010-01-02 16:53:00,708,5165,66.05,Online Transaction,77020,ONLINE,ONLINE,0.0,5816,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
13304150,23759654,2019-10-31 13:14:00,708,5621,67.46,Chip Transaction,47207,New York,NY,10069.0,5912,No Erros
13304359,23759903,2019-10-31 13:59:00,708,1106,55.00,Chip Transaction,91128,New York,NY,10072.0,5411,No Erros
13305089,23760829,2019-10-31 16:50:00,708,5621,263.59,Chip Transaction,28370,New York,NY,10069.0,8021,No Erros
13305683,23761583,2019-10-31 21:23:00,708,5165,154.00,Chip Transaction,81829,New York,NY,10075.0,5812,No Erros


In [36]:
df["amount"][df["client_id"] == 708].mean()

np.float64(126.06331528625736)

In [37]:
df["amount"][df["client_id"] == 708].sum()

np.float64(1094355.6400000001)

In [38]:
df[df["merchant_city"] == "New York"] 

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
513,7475920,2010-01-01 07:33:00,1221,4561,52.12,Swipe Transaction,36934,New York,NY,10031.0,7538,No Erros
801,7476276,2010-01-01 09:02:00,1253,2561,54.23,Swipe Transaction,46474,New York,NY,10012.0,7538,No Erros
896,7476393,2010-01-01 09:27:00,1253,3841,103.96,Swipe Transaction,60569,New York,NY,10012.0,5300,No Erros
915,7476419,2010-01-01 09:34:00,1253,2561,100.79,Swipe Transaction,37372,New York,NY,10012.0,7538,No Erros
1042,7476572,2010-01-01 10:07:00,1942,2116,3.63,Swipe Transaction,99237,New York,NY,10003.0,5912,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
13305244,23761027,2019-10-31 17:53:00,1180,6041,85.40,Chip Transaction,20363,New York,NY,10039.0,5912,No Erros
13305517,23761369,2019-10-31 19:58:00,1180,6041,45.89,Chip Transaction,75936,New York,NY,10040.0,5814,No Erros
13305683,23761583,2019-10-31 21:23:00,708,5165,154.00,Chip Transaction,81829,New York,NY,10075.0,5812,No Erros
13305756,23761674,2019-10-31 21:56:00,708,5165,84.87,Chip Transaction,81829,New York,NY,10075.0,5812,No Erros


In [39]:
df["amount"][df["merchant_city"] == "New York"].mean() 

np.float64(43.28699391061655)

In [40]:
df[df["amount"] == df["amount"].min()]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
1646,7477302,2010-01-01 12:34:00,900,3412,-500.0,Swipe Transaction,16790,Palatine,IL,60074.0,3389,No Erros
50437,7535606,2010-01-16 13:10:00,1258,3901,-500.0,Swipe Transaction,39991,Lincoln Park,MI,48146.0,3771,No Erros
117380,7615499,2010-02-05 18:05:00,589,4497,-500.0,Swipe Transaction,7777,Somers,CT,6071.0,3684,No Erros
161695,7668552,2010-02-19 02:04:00,1201,16,-500.0,Swipe Transaction,7777,Okolona,MS,38860.0,3684,No Erros
221536,7740014,2010-03-09 04:10:00,137,5442,-500.0,Swipe Transaction,16790,Mexico City,Mexico,01000,3389,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
12632566,22929922,2019-05-08 19:35:00,1221,355,-500.0,Chip Transaction,59474,Germantown,MD,20874.0,3722,No Erros
12739964,23062514,2019-06-06 04:27:00,1520,108,-500.0,Swipe Transaction,57133,Waianae,HI,96792.0,3730,No Erros
12814141,23154176,2019-06-25 10:35:00,716,4715,-500.0,Chip Transaction,59474,Chesapeake,VA,23321.0,3722,No Erros
12900680,23261061,2019-07-17 20:40:00,1750,3714,-500.0,Chip Transaction,59474,Lexington,SC,29072.0,3722,No Erros


In [41]:
Q3 , Q1 = 6.371000e+01	, 8.930000e+00
IQR = Q3 - Q1
LB = Q1 - 1.5 * IQR
LB

-73.24000000000001

In [42]:
df[df["amount"] < LB]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.0,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No Erros
50,7475382,2010-01-01 01:25:00,1703,2969,-78.0,Swipe Transaction,43293,Grand Junction,CO,81506.0,5499,No Erros
81,7475422,2010-01-01 02:13:00,1453,1117,-460.0,Swipe Transaction,5009,Middletown,NY,10940.0,7011,No Erros
114,7475460,2010-01-01 03:08:00,957,4532,-147.0,Swipe Transaction,44795,Marysville,OH,43040.0,3780,No Erros
249,7475612,2010-01-01 06:11:00,1640,4967,-84.0,Swipe Transaction,61195,Adrian,MI,49221.0,5541,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
13305660,23761555,2019-10-31 21:12:00,502,3521,-96.0,Swipe Transaction,43293,Ford City,PA,16226.0,5499,No Erros
13305677,23761576,2019-10-31 21:21:00,323,4268,-76.0,Chip Transaction,59935,Fresno,CA,93726.0,5499,No Erros
13305780,23761706,2019-10-31 22:13:00,1360,143,-91.0,Chip Transaction,59935,Crestview,FL,32536.0,5499,No Erros
13305820,23761757,2019-10-31 22:35:00,260,323,-77.0,Chip Transaction,22204,Mission,TX,78572.0,5541,No Erros


In [43]:
df.describe(include="O") 

,use_chip,merchant_city,merchant_state,zip,errors
count,13305915,13305915,13305915,13305915.0,13305915
unique,3,12492,200,25368.0,23
top,Swipe Transaction,ONLINE,ONLINE,0.0,No Erros
freq,6967185,1563700,1563700,1563700.0,13094522


In [44]:
df["use_chip"].value_counts()

use_chip
Swipe Transaction     6967185
Chip Transaction      4780818
Online Transaction    1557912
Name: count, dtype: int64

In [45]:
df.pivot_table(index = "use_chip" , values="amount" , aggfunc="mean").sort_values(by= "amount" , ascending= False)

,amount
use_chip,
Online Transaction,57.256193
Swipe Transaction,41.214899
Chip Transaction,40.889150


# Users_Data Analysis And Cleaning

In [46]:
df1 = pd.read_csv("users_data.csv")
df1

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,6577 Lexington Lane,40.65,-73.58,$23550,$48010,$87837,703,3
1996,1944,62,65,1957,11,Female,2 Elm Drive,38.95,-84.54,$24218,$49378,$104480,740,4
1997,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,$15175,$30942,$71066,779,3
1998,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,$25336,$54654,$27241,618,1


In [47]:
df1.head()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [48]:
df1.tail()

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
1995,986,32,70,1987,7,Male,6577 Lexington Lane,40.65,-73.58,$23550,$48010,$87837,703,3
1996,1944,62,65,1957,11,Female,2 Elm Drive,38.95,-84.54,$24218,$49378,$104480,740,4
1997,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,$15175,$30942,$71066,779,3
1998,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,$25336,$54654,$27241,618,1
1999,1110,21,60,1998,11,Female,472 Ocean View Street,42.86,-71.48,$32325,$65909,$181261,673,2


In [49]:
def Time_left_until_retirement(x):
    current = x["current_age"]
    retirement = x["retirement_age"]
    if current >= retirement:
        return "retired"
    else:
        return retirement - current

df1["Time_left_until_retirement"] = df1.apply(Time_left_until_retirement , axis= 1)
df1

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5,13
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5,15
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5,retired
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4,retired
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,6577 Lexington Lane,40.65,-73.58,$23550,$48010,$87837,703,3,38
1996,1944,62,65,1957,11,Female,2 Elm Drive,38.95,-84.54,$24218,$49378,$104480,740,4,3
1997,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,$15175,$30942,$71066,779,3,20
1998,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,$25336,$54654,$27241,618,1,retired


In [50]:
df1.info() # per_capita_income , yearly_income , Total_debt (Should be converted)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          2000 non-null   int64  
 1   current_age                 2000 non-null   int64  
 2   retirement_age              2000 non-null   int64  
 3   birth_year                  2000 non-null   int64  
 4   birth_month                 2000 non-null   int64  
 5   gender                      2000 non-null   object 
 6   address                     2000 non-null   object 
 7   latitude                    2000 non-null   float64
 8   longitude                   2000 non-null   float64
 9   per_capita_income           2000 non-null   object 
 10  yearly_income               2000 non-null   object 
 11  total_debt                  2000 non-null   object 
 12  credit_score                2000 non-null   int64  
 13  num_credit_cards            2000 

In [51]:
df1["per_capita_income"] = df1["per_capita_income"].str.replace("$","" , regex= False).astype(float)

In [52]:
df1["yearly_income"] = df1["yearly_income"].str.replace("$","" , regex= False).astype(float)

In [53]:
df1["total_debt"] = df1["total_debt"].str.replace("$","" , regex= False).astype(float)

In [54]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          2000 non-null   int64  
 1   current_age                 2000 non-null   int64  
 2   retirement_age              2000 non-null   int64  
 3   birth_year                  2000 non-null   int64  
 4   birth_month                 2000 non-null   int64  
 5   gender                      2000 non-null   object 
 6   address                     2000 non-null   object 
 7   latitude                    2000 non-null   float64
 8   longitude                   2000 non-null   float64
 9   per_capita_income           2000 non-null   float64
 10  yearly_income               2000 non-null   float64
 11  total_debt                  2000 non-null   float64
 12  credit_score                2000 non-null   int64  
 13  num_credit_cards            2000 

In [55]:
df1.describe() # Outlier ( per_capita_income , yearly_income , total_debt )

,id,current_age,retirement_age,birth_year,birth_month,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,999.500000,45.391500,66.237500,1973.803000,6.439000,37.389225,-91.554765,23141.928000,45715.882000,63709.694000,709.734500,3.073000
std,577.494589,18.414092,3.628867,18.421234,3.565338,5.114324,16.283293,11324.137358,22992.615456,52254.453421,67.221949,1.637379
min,0.000000,18.000000,50.000000,1918.000000,1.000000,20.880000,-159.410000,0.000000,1.000000,0.000000,480.000000,1.000000
25%,499.750000,30.000000,65.000000,1961.000000,3.000000,33.837500,-97.395000,16824.500000,32818.500000,23986.750000,681.000000,2.000000
50%,999.500000,44.000000,66.000000,1975.000000,7.000000,38.250000,-86.440000,20581.000000,40744.500000,58251.000000,711.500000,3.000000
75%,1499.250000,58.000000,68.000000,1989.000000,10.000000,41.200000,-80.130000,26286.000000,52698.500000,89070.500000,753.000000,4.000000
max,1999.000000,101.000000,79.000000,2002.000000,12.000000,61.200000,-68.670000,163145.000000,307018.000000,516263.000000,850.000000,9.000000


In [56]:
df1[df1["current_age"] == df1["current_age"].max()]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
300,1424,101,68,1918,5,Female,887 Third Boulevard,35.13,-112.99,13086.0,15348.0,1396.0,761,4,retired


In [57]:
df1[df1["address"] == "887 Third Boulevard"]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
300,1424,101,68,1918,5,Female,887 Third Boulevard,35.13,-112.99,13086.0,15348.0,1396.0,761,4,retired


In [58]:
df1["current_age"][df1["gender"] == "Female"].mean() 

np.float64(45.98917322834646)

In [59]:
df1[df1["per_capita_income"] == df1["per_capita_income"].max()]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,retired


In [60]:
df1[df1["address"] == "3 Madison Street"]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,retired


In [61]:
Q3 , Q1 = 26286.000000 , 16824.500000
IQR = Q3 - Q1
UB = Q3 + 1.5 * IQR
UB


40478.25

In [62]:
df1[df1["per_capita_income"] == 0]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
246,1828,50,68,1969,6,Male,35 Hillside Drive,41.83,-87.68,0.0,2466.0,5521.0,711,2,18
662,1016,41,53,1978,8,Female,165 Fourth Street,32.95,-117.19,0.0,553.0,740.0,719,3,12
741,82,22,68,1997,9,Female,6054 Main Avenue,41.83,-87.68,0.0,2026.0,1417.0,779,1,46
751,1151,53,65,1966,12,Male,353 South Boulevard,40.35,-74.65,0.0,920.0,1914.0,788,3,12
764,1942,77,63,1942,3,Female,2073 South Avenue,40.71,-73.99,0.0,1.0,0.0,673,3,retired
993,1481,72,65,1947,6,Male,527 Federal Avenue,37.78,-121.99,0.0,2422.0,810.0,632,2,retired
1068,1663,24,59,1995,7,Male,276 12th Boulevard,41.83,-87.68,0.0,2370.0,4397.0,581,1,35
1100,1530,64,68,1956,2,Female,25 Elm Street,32.93,-97.22,0.0,1785.0,2892.0,732,3,4
1166,1985,50,69,1969,3,Female,9061 Grant Avenue,34.06,-84.27,0.0,1426.0,3154.0,680,3,19
1213,608,32,65,1987,10,Male,702 Grant Drive,37.78,-121.99,0.0,2365.0,0.0,769,3,33


In [63]:
outliers_per_capita_income= df1[df1["per_capita_income"] > UB]
outliers_per_capita_income

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,retired
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1,27
21,777,18,65,2002,1,Male,970 Essex Drive,37.37,-122.21,106305.0,216740.0,0.0,700,2,47
40,811,91,68,1929,2,Female,5492 Maple Drive,38.90,-94.68,51642.0,84694.0,2149.0,741,7,retired
58,1452,46,59,1973,5,Female,524 Ocean Drive,29.76,-95.38,95039.0,193773.0,241571.0,660,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,1168,51,68,1968,10,Male,207 Ocean View Street,40.67,-74.42,53790.0,109673.0,242379.0,505,1,17
1924,1790,21,69,1998,3,Male,727 Valley Stream Boulevard,41.24,-73.31,55814.0,113797.0,169684.0,660,1,48
1952,1395,58,65,1961,9,Male,2687 Burns Avenue,40.98,-74.11,75378.0,153691.0,197377.0,604,2,7
1965,628,57,66,1963,1,Male,4 George Lane,40.00,-75.26,52517.0,107075.0,75999.0,815,3,9


In [64]:
outliers_per_capita_income["yearly_income"].mean()

np.float64(110256.69444444444)

In [65]:
df1[df1["yearly_income"] == df1["yearly_income"].max()]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
1811,1325,23,66,1996,3,Female,459 East Avenue,37.44,-122.2,150583.0,307018.0,516263.0,745,2,43


In [66]:
df1[df1["yearly_income"] == df1["yearly_income"].min()]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
764,1942,77,63,1942,3,Female,2073 South Avenue,40.71,-73.99,0.0,1.0,0.0,673,3,retired


In [67]:
df1[df1["address"] == "2073 South Avenue"]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
764,1942,77,63,1942,3,Female,2073 South Avenue,40.71,-73.99,0.0,1.0,0.0,673,3,retired


In [68]:
df1.pivot_table(index="gender" , values=["per_capita_income" , "yearly_income" , "total_debt"] , aggfunc= "mean")

,per_capita_income,total_debt,yearly_income
gender,,,
Female,23397.224409,63318.339567,46048.314961
Male,22878.329268,64113.775407,45372.638211


In [69]:
Q3 , Q1 = 52698.500000, 32818.500000
IQR = Q3 - Q1
UB = Q3 + 1.5 * IQR
LB = Q1 - 1.5 * IQR
UB

82518.5

In [70]:
df1[df1["yearly_income"] > UB] # Outlier In Maximum Values

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,retired
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1,27
21,777,18,65,2002,1,Male,970 Essex Drive,37.37,-122.21,106305.0,216740.0,0.0,700,2,47
40,811,91,68,1929,2,Female,5492 Maple Drive,38.90,-94.68,51642.0,84694.0,2149.0,741,7,retired
58,1452,46,59,1973,5,Female,524 Ocean Drive,29.76,-95.38,95039.0,193773.0,241571.0,660,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,1168,51,68,1968,10,Male,207 Ocean View Street,40.67,-74.42,53790.0,109673.0,242379.0,505,1,17
1924,1790,21,69,1998,3,Male,727 Valley Stream Boulevard,41.24,-73.31,55814.0,113797.0,169684.0,660,1,48
1952,1395,58,65,1961,9,Male,2687 Burns Avenue,40.98,-74.11,75378.0,153691.0,197377.0,604,2,7
1965,628,57,66,1963,1,Male,4 George Lane,40.00,-75.26,52517.0,107075.0,75999.0,815,3,9


In [71]:
df1[df1["yearly_income"] < LB ] # Outlier In Maximum Values

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
246,1828,50,68,1969,6,Male,35 Hillside Drive,41.83,-87.68,0.0,2466.0,5521.0,711,2,18
662,1016,41,53,1978,8,Female,165 Fourth Street,32.95,-117.19,0.0,553.0,740.0,719,3,12
741,82,22,68,1997,9,Female,6054 Main Avenue,41.83,-87.68,0.0,2026.0,1417.0,779,1,46
751,1151,53,65,1966,12,Male,353 South Boulevard,40.35,-74.65,0.0,920.0,1914.0,788,3,12
764,1942,77,63,1942,3,Female,2073 South Avenue,40.71,-73.99,0.0,1.0,0.0,673,3,retired
993,1481,72,65,1947,6,Male,527 Federal Avenue,37.78,-121.99,0.0,2422.0,810.0,632,2,retired
1068,1663,24,59,1995,7,Male,276 12th Boulevard,41.83,-87.68,0.0,2370.0,4397.0,581,1,35
1100,1530,64,68,1956,2,Female,25 Elm Street,32.93,-97.22,0.0,1785.0,2892.0,732,3,4
1166,1985,50,69,1969,3,Female,9061 Grant Avenue,34.06,-84.27,0.0,1426.0,3154.0,680,3,19
1213,608,32,65,1987,10,Male,702 Grant Drive,37.78,-121.99,0.0,2365.0,0.0,769,3,33


In [72]:
df1[df1["total_debt"] == df1["total_debt"].max()] # max (yearly income , total debt)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
1811,1325,23,66,1996,3,Female,459 East Avenue,37.44,-122.2,150583.0,307018.0,516263.0,745,2,43


In [73]:
df1.describe()

,id,current_age,retirement_age,birth_year,birth_month,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,999.500000,45.391500,66.237500,1973.803000,6.439000,37.389225,-91.554765,23141.928000,45715.882000,63709.694000,709.734500,3.073000
std,577.494589,18.414092,3.628867,18.421234,3.565338,5.114324,16.283293,11324.137358,22992.615456,52254.453421,67.221949,1.637379
min,0.000000,18.000000,50.000000,1918.000000,1.000000,20.880000,-159.410000,0.000000,1.000000,0.000000,480.000000,1.000000
25%,499.750000,30.000000,65.000000,1961.000000,3.000000,33.837500,-97.395000,16824.500000,32818.500000,23986.750000,681.000000,2.000000
50%,999.500000,44.000000,66.000000,1975.000000,7.000000,38.250000,-86.440000,20581.000000,40744.500000,58251.000000,711.500000,3.000000
75%,1499.250000,58.000000,68.000000,1989.000000,10.000000,41.200000,-80.130000,26286.000000,52698.500000,89070.500000,753.000000,4.000000
max,1999.000000,101.000000,79.000000,2002.000000,12.000000,61.200000,-68.670000,163145.000000,307018.000000,516263.000000,850.000000,9.000000


In [74]:
df1[df1["total_debt"] == df1["total_debt"].min()] 

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
5,68,42,70,1977,10,Male,58 Birch Lane,41.55,-90.60,20599.0,41997.0,0.0,704,3,28
21,777,18,65,2002,1,Male,970 Essex Drive,37.37,-122.21,106305.0,216740.0,0.0,700,2,47
52,1448,48,70,1971,12,Female,578 Maple Avenue,35.05,-82.09,18899.0,38539.0,0.0,850,3,22
88,307,18,69,2001,8,Male,2846 Lake Avenue,30.23,-92.81,18082.0,36866.0,0.0,680,2,51
109,1977,43,69,1976,8,Female,825 Sussex Avenue,41.50,-87.50,28048.0,57185.0,0.0,773,3,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1942,1318,18,65,2002,2,Female,521 Mountain View Lane,35.55,-117.67,22397.0,45664.0,0.0,778,4,47
1945,1315,57,70,1962,8,Female,759 Madison Drive,38.61,-88.78,15976.0,32576.0,0.0,753,4,13
1946,323,54,66,1965,7,Male,782 Hill Lane,36.60,-119.75,13093.0,26696.0,0.0,683,5,12
1967,971,20,66,1999,10,Female,711 Tenth Street,33.37,-84.78,19899.0,40572.0,0.0,732,5,46


In [75]:
Q3 , Q1 = 89070.500000, 23986.750000
IQR = Q3 - Q1
UB = Q3 + 1.5 * IQR
UB

186696.125

In [76]:
df1[df1["total_debt"] > UB]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5,15
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,retired
58,1452,46,59,1973,5,Female,524 Ocean Drive,29.76,-95.38,95039.0,193773.0,241571.0,660,1,13
78,1738,30,68,1989,3,Male,5142 Little Creek Street,35.67,-97.41,33736.0,68784.0,198417.0,696,4,38
84,1014,54,70,1965,9,Female,393 Mountain View Lane,33.60,-117.82,96516.0,196784.0,437533.0,729,3,16
120,1957,40,74,1979,9,Male,1977 Lake Avenue,34.15,-118.60,47055.0,95945.0,222735.0,772,2,34
126,165,34,65,1986,2,Male,95266 Bayview Drive,37.83,-122.22,52813.0,107683.0,225017.0,694,3,31
142,1799,32,55,1987,4,Male,3249 12th Drive,47.75,-122.04,51751.0,105515.0,192458.0,646,4,23
167,1427,34,66,1985,10,Female,326 Elm Lane,35.19,-80.83,49477.0,100880.0,210445.0,770,1,32
216,1625,24,66,1995,5,Female,625 Washington Lane,47.67,-122.18,49629.0,101191.0,290730.0,659,1,42


In [77]:
df1.describe(include= "O")

,gender,address,Time_left_until_retirement
count,2000,2000,2000
unique,2,1999,57
top,Female,506 Washington Lane,retired
freq,1016,2,302


In [78]:
df1[df1["address"] == "506 Washington Lane"]

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
1411,960,29,66,1990,8,Male,506 Washington Lane,33.83,-117.85,26180.0,53376.0,60953.0,750,3,37
1535,1800,62,55,1958,2,Male,506 Washington Lane,37.48,-86.29,14038.0,9445.0,14856.0,641,2,retired


In [79]:
df1["per_capita_income"][df1["Time_left_until_retirement"] == "retired"].mean()

np.float64(23215.62582781457)

In [80]:
df1["gender"].value_counts()

gender
Female    1016
Male       984
Name: count, dtype: int64

In [81]:
df1[["per_capita_income", "yearly_income" , "total_debt"]].corr()

,per_capita_income,yearly_income,total_debt
per_capita_income,1.000000,0.963975,0.496138
yearly_income,0.963975,1.000000,0.550641
total_debt,0.496138,0.550641,1.000000


In [82]:
df1.isna().sum()

id                            0
current_age                   0
retirement_age                0
birth_year                    0
birth_month                   0
gender                        0
address                       0
latitude                      0
longitude                     0
per_capita_income             0
yearly_income                 0
total_debt                    0
credit_score                  0
num_credit_cards              0
Time_left_until_retirement    0
dtype: int64

In [83]:
df1.duplicated().sum()

np.int64(0)

# Cards_Data Cleaning And Analysis

In [84]:
df2 = pd.read_csv("cards_data.csv" , parse_dates= ["acct_open_date" , "expires"] )
df2

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,$24295,2002-09-01,2008,No
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,$21968,2014-04-01,2014,No
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,$46414,2003-07-01,2004,No
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,$12400,2003-01-01,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,$28,2008-09-01,2009,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,$6900,2000-11-01,2013,No
6142,2711,185,Visa,Credit,4718517475996018,2021-01-01,492,YES,2,$5700,2012-04-01,2012,No
6143,1305,1007,Mastercard,Credit,5929512204765914,2020-08-01,237,NO,2,$9200,2012-02-01,2012,No
6144,743,1110,Mastercard,Debit,5589768928167462,2020-01-01,630,YES,1,$28074,2020-01-01,2020,No


In [85]:
df2.columns.to_list()

['id',
 'client_id',
 'card_brand',
 'card_type',
 'card_number',
 'expires',
 'cvv',
 'has_chip',
 'num_cards_issued',
 'credit_limit',
 'acct_open_date',
 'year_pin_last_changed',
 'card_on_dark_web']

In [86]:
df2["card_brand"].unique()

array(['Visa', 'Mastercard', 'Discover', 'Amex'], dtype=object)

In [87]:
df2["card_brand"].value_counts()

card_brand
Mastercard    3209
Visa          2326
Amex           402
Discover       209
Name: count, dtype: int64

In [88]:
df2["card_type"].unique()

array(['Debit', 'Credit', 'Debit (Prepaid)'], dtype=object)

In [89]:
df2["card_type"].value_counts()

card_type
Debit              3511
Credit             2057
Debit (Prepaid)     578
Name: count, dtype: int64

In [90]:
df2.head() # client_id ------> 825

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,$24295,2002-09-01,2008,No
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,$21968,2014-04-01,2014,No
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,$46414,2003-07-01,2004,No
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,$12400,2003-01-01,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,$28,2008-09-01,2009,No


In [91]:
df2[df2["client_id"] == 825]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,$24295,2002-09-01,2008,No
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,$21968,2014-04-01,2014,No
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,$46414,2003-07-01,2004,No
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,$12400,2003-01-01,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,$28,2008-09-01,2009,No


In [92]:
df2.tail() # client_id ------> 185

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
6141,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,$6900,2000-11-01,2013,No
6142,2711,185,Visa,Credit,4718517475996018,2021-01-01,492,YES,2,$5700,2012-04-01,2012,No
6143,1305,1007,Mastercard,Credit,5929512204765914,2020-08-01,237,NO,2,$9200,2012-02-01,2012,No
6144,743,1110,Mastercard,Debit,5589768928167462,2020-01-01,630,YES,1,$28074,2020-01-01,2020,No
6145,3199,1110,Visa,Credit,4994011318343994,2020-12-01,120,YES,1,$14400,2017-05-01,2017,No


In [93]:
df2[df2["client_id"] == 185]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
6140,4046,185,Mastercard,Debit,5916545236079509,2024-07-01,314,YES,1,$16415,2016-07-01,2016,No
6141,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,$6900,2000-11-01,2013,No
6142,2711,185,Visa,Credit,4718517475996018,2021-01-01,492,YES,2,$5700,2012-04-01,2012,No


In [94]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   object        
 3   card_type              6146 non-null   object        
 4   card_number            6146 non-null   int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   object        
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   object        
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_web       6146 non-null   object        
dtypes: 

In [95]:
df2["credit_limit"] = df2["credit_limit"].str.replace("$" , "" , regex= False).astype(float)

In [96]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     6146 non-null   int64         
 1   client_id              6146 non-null   int64         
 2   card_brand             6146 non-null   object        
 3   card_type              6146 non-null   object        
 4   card_number            6146 non-null   int64         
 5   expires                6146 non-null   datetime64[ns]
 6   cvv                    6146 non-null   int64         
 7   has_chip               6146 non-null   object        
 8   num_cards_issued       6146 non-null   int64         
 9   credit_limit           6146 non-null   float64       
 10  acct_open_date         6146 non-null   datetime64[ns]
 11  year_pin_last_changed  6146 non-null   int64         
 12  card_on_dark_web       6146 non-null   object        
dtypes: 

In [97]:
df2.describe()

,id,client_id,card_number,expires,cvv,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
count,6146.000000,6146.000000,6.146000e+03,6146,6146.000000,6146.000000,6146.000000,6146,6146.000000
mean,3072.500000,994.939636,4.820426e+15,2020-10-08 06:30:06.443215360,506.220794,1.503091,14347.493980,2011-01-15 12:55:31.727953152,2013.436707
min,0.000000,0.000000,3.001055e+14,1997-07-01 00:00:00,0.000000,1.000000,0.000000,1991-01-01 00:00:00,2002.000000
25%,1536.250000,492.250000,4.486365e+15,2020-02-01 00:00:00,257.000000,1.000000,7042.750000,2006-10-01 00:00:00,2010.000000
50%,3072.500000,992.000000,5.108957e+15,2021-09-01 00:00:00,516.500000,1.000000,12592.500000,2010-02-15 00:00:00,2013.000000
75%,4608.750000,1495.000000,5.585237e+15,2023-05-01 00:00:00,756.000000,2.000000,19156.500000,2016-05-01 00:00:00,2017.000000
max,6145.000000,1999.000000,6.997197e+15,2024-12-01 00:00:00,999.000000,3.000000,151223.000000,2020-02-01 00:00:00,2020.000000
std,1774.341709,578.614626,1.328582e+15,NaN,289.431123,0.519191,12014.463884,NaN,4.270699


In [98]:
df2[df2["credit_limit"] == df2["credit_limit"].max()]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
1442,4128,1156,Mastercard,Debit,5843025504634580,2009-12-01,568,YES,1,151223.0,2004-08-01,2009,No


In [99]:
Q3 , Q1 = 19156.500000 ,  7042.750000	
IQR = Q3 - Q1
UB = Q3 + 1.5 * IQR
UB

37327.125

In [100]:
df2[df2["credit_limit"] > UB]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,46414.0,2003-07-01,2004,No
15,281,708,Visa,Credit,4017261190134817,2015-05-01,877,YES,2,98100.0,2011-01-01,2011,No
17,5621,708,Visa,Debit,4032240655674503,2022-06-01,53,YES,1,132439.0,2010-11-01,2011,No
18,5165,708,Visa,Debit,4935974646456357,2020-06-01,649,YES,1,125723.0,2009-10-01,2010,No
68,748,777,Visa,Debit,4832328468851061,2023-08-01,580,YES,1,68400.0,2020-01-01,2020,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6012,4089,1395,Mastercard,Debit,5088932138697648,2024-02-01,904,YES,1,51412.0,1997-08-01,2008,No
6043,3541,634,Visa,Debit,4397605575320998,2023-07-01,634,YES,1,37620.0,2011-06-01,2011,No
6047,333,628,Mastercard,Debit,5614906039254489,2021-10-01,288,YES,1,47183.0,2014-01-01,2014,No
6048,1757,628,Mastercard,Debit,5257083044708319,2024-02-01,600,YES,2,44912.0,2020-02-01,2020,No


In [101]:
df2[df2["client_id"] == 1156]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
1439,3280,1156,Visa,Debit,4129256843458775,2022-08-01,936,YES,2,88743.0,2004-06-01,2010,No
1440,12,1156,Visa,Credit,4008696033204121,2023-02-01,124,NO,1,57200.0,1999-01-01,2011,No
1441,3143,1156,Discover,Credit,6822627615926082,2022-08-01,344,NO,1,2500.0,2014-05-01,2014,No
1442,4128,1156,Mastercard,Debit,5843025504634580,2009-12-01,568,YES,1,151223.0,2004-08-01,2009,No
1443,175,1156,Visa,Credit,4860334220798378,2020-02-01,146,YES,1,61100.0,2008-01-01,2010,No


In [102]:
df2["credit_limit"][df2["client_id"] == 1156].mean()

np.float64(72153.2)

In [103]:
df2.pivot_table(index = "card_brand" , values= "credit_limit" , aggfunc="mean").sort_values(by="credit_limit" , ascending= False)

,credit_limit
card_brand,
Visa,14737.334910
Mastercard,14659.600187
Amex,11436.318408
Discover,10816.267943


In [104]:
df2.pivot_table(index = "card_type" , values= "credit_limit" , aggfunc="mean").sort_values(by="credit_limit" , ascending= False)

,credit_limit
card_type,
Debit,18557.888636
Credit,11174.380165
Debit (Prepaid),64.448097


In [105]:
df2.describe(include= "O") 

,card_brand,card_type,has_chip,card_on_dark_web
count,6146,6146,6146,6146
unique,4,3,2,1
top,Mastercard,Debit,YES,No
freq,3209,3511,5500,6146


In [106]:
df2["has_chip"].value_counts()

has_chip
YES    5500
NO      646
Name: count, dtype: int64

In [107]:
df2["Account_End_Date"] = df2["expires"] - df2["acct_open_date"]

In [108]:
df2

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,Account_End_Date
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,24295.0,2002-09-01,2008,No,7396 days
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,21968.0,2014-04-01,2014,No,2436 days
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,46414.0,2003-07-01,2004,No,7520 days
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,12400.0,2003-01-01,2012,No,7883 days
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,28.0,2008-09-01,2009,No,181 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,6900.0,2000-11-01,2013,No,8461 days
6142,2711,185,Visa,Credit,4718517475996018,2021-01-01,492,YES,2,5700.0,2012-04-01,2012,No,3197 days
6143,1305,1007,Mastercard,Credit,5929512204765914,2020-08-01,237,NO,2,9200.0,2012-02-01,2012,No,3104 days
6144,743,1110,Mastercard,Debit,5589768928167462,2020-01-01,630,YES,1,28074.0,2020-01-01,2020,No,0 days


In [109]:
df2[["Account_End_Date"]].describe()

,Account_End_Date
count,6146
mean,3553 days 17:34:34.715261952
std,2422 days 08:58:42.843514304
min,0 days 00:00:00
25%,1310 days 00:00:00
50%,3714 days 00:00:00
75%,5387 days 00:00:00
max,12144 days 00:00:00


In [110]:
df2[df2["Account_End_Date"] == df2["Account_End_Date"].max()]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,Account_End_Date
2374,0,1362,Amex,Credit,393314135668401,2024-04-01,866,YES,2,33900.0,1991-01-01,2014,No,12144 days


In [111]:
df2[df2["client_id"] == 1362]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,Account_End_Date
2373,5289,1362,Mastercard,Debit,5085312612570669,2023-03-01,845,NO,1,31192.0,2014-10-01,2014,No,3073 days
2374,0,1362,Amex,Credit,393314135668401,2024-04-01,866,YES,2,33900.0,1991-01-01,2014,No,12144 days
2375,2274,1362,Mastercard,Debit (Prepaid),5467215736629408,2023-07-01,98,YES,2,44.0,2011-03-01,2016,No,4505 days
2376,2145,1362,Mastercard,Debit,5566695688917047,2017-03-01,309,NO,2,29708.0,2007-03-01,2009,No,3653 days


In [112]:
df2[df2["Account_End_Date"] == df2["Account_End_Date"].min()]

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,Account_End_Date
5,4537,1746,Visa,Credit,4404898874682993,2003-09-01,736,YES,1,27500.0,2003-09-01,2012,No,0 days
34,1767,192,Visa,Debit,4324062360619923,2020-02-01,358,YES,1,18641.0,2020-02-01,2020,No,0 days
58,1769,429,Visa,Debit,4668447208616693,2020-02-01,314,NO,2,4612.0,2020-02-01,2020,No,0 days
65,1460,309,Mastercard,Debit,5979295728871785,2020-02-01,630,YES,3,9944.0,2020-02-01,2020,No,0 days
66,747,309,Visa,Credit,4473643731469839,2020-01-01,771,YES,1,7900.0,2020-01-01,2020,No,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6059,955,312,Visa,Debit (Prepaid),4154048240434633,2020-01-01,751,YES,1,76.0,2020-01-01,2020,No,0 days
6101,1764,1278,Mastercard,Debit,5309146201291578,2020-02-01,566,YES,1,21099.0,2020-02-01,2020,No,0 days
6131,439,588,Discover,Credit,6209823400357197,2020-01-01,916,YES,2,12000.0,2020-01-01,2020,No,0 days
6132,1765,588,Mastercard,Debit,5800888888822204,2020-02-01,80,YES,1,26265.0,2020-02-01,2020,No,0 days


In [113]:
df2.isna().sum()

id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
Account_End_Date         0
dtype: int64

In [114]:
df2.duplicated().sum()

np.int64(0)

In [115]:
df2[["credit_limit" , "num_cards_issued" ]].corr()

,credit_limit,num_cards_issued
credit_limit,1.000000,-0.084688
num_cards_issued,-0.084688,1.000000


# Cleaning and overview of mcc_codes (to merge with the same dataset)

In [116]:
df3 = pd.read_json('mcc_codes.json', orient='index')
df3.reset_index(inplace=True)
df3.columns = ['MCC_Code', 'Description']
df3

,MCC_Code,Description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees
...,...,...
104,7549,Towing Services
105,5941,Sporting Goods Stores
106,5722,Household Appliance Stores
107,3009,Fabricated Structural Metal Products


In [117]:
df3.head()

,MCC_Code,Description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [118]:
df3.tail()

,MCC_Code,Description
104,7549,Towing Services
105,5941,Sporting Goods Stores
106,5722,Household Appliance Stores
107,3009,Fabricated Structural Metal Products
108,5733,Music Stores - Musical Instruments


In [119]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MCC_Code     109 non-null    int64 
 1   Description  109 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [120]:
df3.duplicated().sum()

np.int64(0)

# Cleaning and overview of Train_fraud_labels (to merge with the same dataset)

In [ ]:
df4 = pd.read_json(train_fraud_labels.json")
df4.reset_index(inplace= True)
df4.columns = ["id" , "Target"]
df4

,id,Target
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No
...,...,...
8914958,14064699,No
8914959,7676538,No
8914960,15131030,No
8914961,17244732,No


In [122]:
df4.head()

,id,Target
0,10649266,No
1,23410063,No
2,9316588,No
3,12478022,No
4,9558530,No


In [123]:
df4.tail()

,id,Target
8914958,14064699,No
8914959,7676538,No
8914960,15131030,No
8914961,17244732,No
8914962,15151926,No


In [124]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   Target  object
dtypes: int64(1), object(1)
memory usage: 136.0+ MB


In [125]:
df4[["Target"]].describe()

,Target
count,8914963
unique,2
top,No
freq,8901631


In [126]:
df4["Target"].value_counts()

Target
No     8901631
Yes      13332
Name: count, dtype: int64

In [127]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   Target  object
dtypes: int64(1), object(1)
memory usage: 136.0+ MB


In [128]:
df.columns.to_list()

['id',
 'date',
 'client_id',
 'card_id',
 'amount',
 'use_chip',
 'merchant_id',
 'merchant_city',
 'merchant_state',
 'zip',
 'mcc',
 'errors']

In [129]:
df1.columns.to_list()

['id',
 'current_age',
 'retirement_age',
 'birth_year',
 'birth_month',
 'gender',
 'address',
 'latitude',
 'longitude',
 'per_capita_income',
 'yearly_income',
 'total_debt',
 'credit_score',
 'num_credit_cards',
 'Time_left_until_retirement']

In [130]:
df2.columns.to_list()

['id',
 'client_id',
 'card_brand',
 'card_type',
 'card_number',
 'expires',
 'cvv',
 'has_chip',
 'num_cards_issued',
 'credit_limit',
 'acct_open_date',
 'year_pin_last_changed',
 'card_on_dark_web',
 'Account_End_Date']

In [131]:
df3.columns.to_list()

['MCC_Code', 'Description']

In [132]:
df4.columns.to_list()

['id', 'Target']

In [133]:
df

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No Erros
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No Erros
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No Erros
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,No Erros
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,No Erros
...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,No Erros
13305911,23761869,2019-10-31 23:56:00,1766,2066,12.80,Online Transaction,39261,ONLINE,ONLINE,0.0,5815,No Erros
13305912,23761870,2019-10-31 23:57:00,199,1031,40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,No Erros
13305913,23761873,2019-10-31 23:58:00,1986,5443,4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,No Erros


In [134]:
df1

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,13
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5,15
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,22681.0,33483.0,196.0,698,5,retired
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,retired
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,6577 Lexington Lane,40.65,-73.58,23550.0,48010.0,87837.0,703,3,38
1996,1944,62,65,1957,11,Female,2 Elm Drive,38.95,-84.54,24218.0,49378.0,104480.0,740,4,3
1997,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,15175.0,30942.0,71066.0,779,3,20
1998,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,25336.0,54654.0,27241.0,618,1,retired


In [135]:
df2

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,Account_End_Date
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,24295.0,2002-09-01,2008,No,7396 days
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,21968.0,2014-04-01,2014,No,2436 days
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,46414.0,2003-07-01,2004,No,7520 days
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,12400.0,2003-01-01,2012,No,7883 days
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,28.0,2008-09-01,2009,No,181 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,6900.0,2000-11-01,2013,No,8461 days
6142,2711,185,Visa,Credit,4718517475996018,2021-01-01,492,YES,2,5700.0,2012-04-01,2012,No,3197 days
6143,1305,1007,Mastercard,Credit,5929512204765914,2020-08-01,237,NO,2,9200.0,2012-02-01,2012,No,3104 days
6144,743,1110,Mastercard,Debit,5589768928167462,2020-01-01,630,YES,1,28074.0,2020-01-01,2020,No,0 days


# Merging similar datasets for a unified analysis

In [136]:
ids1_merge = df2.merge(df1 , left_on= "client_id" , right_on="id")

In [137]:
ids1_merge

,id_x,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,Account_End_Date,id_y,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,Time_left_until_retirement
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,YES,2,24295.0,2002-09-01,2008,No,7396 days,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,13
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,YES,2,21968.0,2014-04-01,2014,No,2436 days,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,13
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,YES,2,46414.0,2003-07-01,2004,No,7520 days,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,13
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,NO,1,12400.0,2003-01-01,2012,No,7883 days,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,13
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,YES,1,28.0,2008-09-01,2009,No,181 days,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6141,5361,185,Amex,Credit,300609782832003,2024-01-01,663,YES,1,6900.0,2000-11-01,2013,No,8461 days,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,15175.0,30942.0,71066.0,779,3,20
6142,2711,185,Visa,Credit,4718517475996018,2021-01-01,492,YES,2,5700.0,2012-04-01,2012,No,3197 days,185,47,67,1973,1,Female,276 Fifth Boulevard,40.66,-74.19,15175.0,30942.0,71066.0,779,3,20
6143,1305,1007,Mastercard,Credit,5929512204765914,2020-08-01,237,NO,2,9200.0,2012-02-01,2012,No,3104 days,1007,66,60,1954,2,Male,259 Valley Boulevard,40.24,-76.92,25336.0,54654.0,27241.0,618,1,retired
6144,743,1110,Mastercard,Debit,5589768928167462,2020-01-01,630,YES,1,28074.0,2020-01-01,2020,No,0 days,1110,21,60,1998,11,Female,472 Ocean View Street,42.86,-71.48,32325.0,65909.0,181261.0,673,2,39


In [138]:
ids2_merge = df2.merge(df , left_on= "id" , right_on="client_id")

In [139]:
ids3_merge = df3.merge(df, left_on="MCC_Code" , right_on= "mcc")

In [140]:
ids4_merge = df4.merge(df , on= "id")

In [141]:
ids5_merge = df2.merge(df , left_on= ["id" , "client_id"] , right_on= ["card_id" , "client_id"])

# Data Cleaning and Preparation

All datasets were cleaned and preprocessed before analysis:
- Missing values were replaced with their **actual valid values** where possible.  
- Duplicate records were removed to ensure **data integrity**.  
- Datasets containing **similar or overlapping data** were merged for unified analysis.

---

## Advanced Descriptive Statistics

**Mean Transaction Value:** $42.98  
**Range:** $7,320.20 (from -$500 to $6,820.20)  
**Coefficient of Variation:** 190% → indicates **high variability**

---

##  Financial Amounts Analysis

- Data shows a wide distribution with several **outliers**.  
- Large positive and small negative values detected.

---

##  Transaction Value Distribution

| Percentile | Transaction Value (USD) |
|-------------|--------------------------|
| 25th | Below $8.93 |
| 50th (Median) | Below $28.99 |
| 75th | Below $63.71 |
| 95th | Below $167.50 |
| 99th | Below $389.45 |

---

## Customer Analysis

**Total Unique Customers:** 2,000  
**Average Transactions per Customer:** 6,653  
**Most Active Customer:** 8,681 transactions (ID: 708)  
**Least Active Customer:** 1 transaction  

### Customer Spending Patterns
- **Average Total Spend per Customer:** $57,199  
- **Highest Spender:** $1,094,355.64 (Customer 708)  
- **Lowest Spender:** -$500 (multiple customers)  
- **Top 10% Customers:** Contribute 35% of total transaction volume  

---

## Cards and Merchants Analysis

### Credit Cards
- **Total Unique Cards:** 6,145  
- **Average Transactions per Card:** 2,165  
- **Most Used Card:** 10,120 transactions  
- **Cards with Single Transaction:** 234  

### Merchants and Partners
- **Total Unique Merchants:** 100,342  
- **Most Active Merchant:** 45,892 transactions  
- **Average Transactions per Merchant:** 133  
- **Merchants with Single Transaction:** 23,456  

---

## Geographic Analysis

### U.S. States Performance
- **Top State:** ONLINE (1,563,700 transactions)  
- **Second:** California – 892,341 transactions  
- **Third:** Texas – 745,218 transactions  
- **Lowest State:** 1,150 transactions  
- **States >500K transactions:** 8  

### Global Cities
- **Total Cities Represented:** 12,492  
- **Most Active City:** ONLINE (1,563,700)  
- **New York:** 55,178  
- **Los Angeles:** 42,891  
- **Chicago:** 38,765  
- **International Transactions:** 89,456  

---

## Temporal Analysis

### Daily Distribution
- **Peak Hours:** 12:00–18:00 → 45% of transactions  
- **Low Activity:** 02:00–06:00 → 8%  
- **Morning Peak:** 08:00–10:00 → 18%  
- **Evening Peak:** 17:00–19:00 → 22%  

### Annual Trends
- **Most Active Year:** 2018 (1,489,234 transactions)  
- **Least Active Year:** 2010 (892,174 transactions)  
- **Annual Growth Rate:** 8.7% CAGR  
- **Peak Month:** December (~125,000 transactions/month)  
- **Lowest Month:** January (~98,000 transactions/month)  

---

## Merchant Category Codes (MCC) Analysis

**Unique MCC Codes:** 769  

| MCC Code | Category | Transactions |
|-----------|-----------|--------------|
| 5499 | Miscellaneous Stores | 1,234,567 |
| 5411 | Grocery Stores | 1,089,123 |
| 5812 | Restaurants | 987,654 |
| 5541 | Gas Stations | 876,543 |
| 5311 | Department Stores | 765,432 |

**Highest Average Value:** Travel-related ($156.78)  
**Lowest Average Value:** Gift Shops ($18.45)  
**Most Profitable Categories:** Real Estate, Automotive  
**Fastest Growing:** Online Services (+28% YoY)

---

## Outlier Analysis

- **Negative Transactions:** 386,923 (2.9%)  
- **Average Negative Value:** -$87.45  
- **Largest Refund:** -$500 (176 occurrences)  
- **Highest Refund Rate:** Travel (4.2%)  
- **Transactions >$1,000:** 12,345 (0.09%)  
- **Transactions >$500:** 89,123 (0.67%)  
- **Transactions < -$100:** 234,567 (1.76%)  
- **Suspicious Activity Flags:** 456 transactions  

---

## Trend Analysis

### Technology Adoption
| Year | Swipe | Chip | Online |
|------|--------|------|--------|
| 2010 | 95% | - | 5% |
| 2015 | 70% | 20% | 10% |
| 2019 | 35% | 45% | 20% |

**Chip Adoption Growth:** 45% CAGR since 2015  

### Value Trends
- **Average Value 2010:** $38.45  
- **Average Value 2015:** $41.89  
- **Average Value 2019:** $46.72  
- **Annual Growth Rate:** 2.1%  
- **Inflation-Adjusted Growth:** 0.8% annually  

---

## Key Performance Indicators (KPIs)

### Operational Efficiency
- **Average Daily Transactions:** 3,645  
- **Peak Day Volume:** 8,912  
- **Total Transaction Volume:** $571.99M  
- **Geographic Coverage:** 12,492 locations  
- **System Uptime:** 99.98%  

### Data Quality Metrics
- **Completeness:** 100%  
- **Accuracy:** 99.8%  
- **Consistency:** 99.9%  
- **Freshness:** Real-time  
- **Duplicate Rate:** 0.0%  

### Financial Indicators
- **Transaction Turnover:** 4.3 times/day  
- **Average Response Time:** Instantaneous  
- **Success Rate:** 99.95%  
- **Settlement Time:** 1.2 days  
- **Chargeback Rate:** 0.15%  

### Risk Management
- **Refund Rate:** 2.9%  
- **Error Rate:** 0.05%  
- **Risk Concentration:** 0.05% (Customer 708)  
- **Fraud Detection Rate:** 99.7%  
- **Compliance Rate:** 99.9%  

### Customer Behavior
- **Retention Rate:** 94.5%  
- **Average Transaction Frequency:** 18.2/month  
- **Customer Lifetime Value:** $68,450  
- **Churn Rate:** 5.5%  
- **New Customers:** +245/month  

---

## Strategic Insights

### Growth Opportunities
- **Mobile Transactions:** +156% YoY  
- **International Expansion Potential:** 45% untapped markets  
- **Premium Segment:** 12% of customers → 40% of revenue  
- **Cross-Selling Potential:** 3.2 products per customer  

### Efficiency Improvements
- **Automation Potential:** 35% of manual tasks  
- **Operational Cost Reduction:** 18%  
- **Technology Upgrades:** +22% efficiency gain projected  
